# WMIR practice lesson on Spacy

### Obiettivi
- Usare il framework Spacy per estrarre informazioni rilevanti dalle frasi per arricchire le loro rappresentazioni.
- Addestra diversi modelli e confrontali con e senza la rappresentazioni "arricchita"  

#### Author
Claudiu Daniel Hromei, April 2023.  
hromei@ing.uniroma2.it

# Introduzione

[SpaCy](spacy.io) è una libreria open-source gratuita per il Natural Language Processing (NLP) in Python. Può essere usata per costruire sistemi di Information Extractrion o di Comprensione del Linguaggio Naturale, o anche per fare preprocessamento di testo per il Deep Learning.  
Alcune caratteristiche di SpaCy sono:
- **Tokenization**: suddividere il testo in unità indivisibili (**token**), formate dalle singole parole, segni di punteggiatura, simboli come 35% ecc.
- **Part-of-speech (POS) Tagging**: Assegnare i tipi delle parole (nome, verbo) ai token.
- **Dependency Parsing**: Assegnare delle etichette di dipendenza sintattica che descrivono le relazioni tra i diversi token. Ad esempio un token è oggetto mentre un altro è il soggetto.
- **Lemmatization**: Convertire le parole nella loro forma base, in italiano nel lemma. Ad esempio la forma all'infinto dei verbi oppure la forma maschile singolare dei nomi. In inglese, il lemma di “was” è “be”, e il lemma di “rats” è “rat”.
- **Sentence Boundary Detection (SBD)**: Trovare e suddividere i testi nelle diverse frasi.
- **Named Entity Recognition (NER)**: Trovare ed etichettare gli oggetti del mondo reale che hanno un nome, ad esempio persone, compagnie o luoghi. "New York City", "Giovanni" sono delle Named Entities.
- **Entity Linking (EL)**: Disambiguare le entità testuali in modo da assegnargli degli identificatori univoci in una base di conoscenza.
- **Similarity**: Confrontare le parole, testi e documenti e fornire una misura di similarità tra loro.
- **Text Classification**: Assegnare delle categorie o etichette a interi documenti o parti di esso.
- **Rule-based Matching**: Trovare le sequenze di token in base alle loro annotazioni linguistiche, in modo simile alle Regular Expression.
- **Training**: Aggiornare e migliorare le predizioni di un modello statistico
- **Serialization**: Salvare gli oggetti su file o su stringhe di bytes.

# Required Libraries

In [1]:
import pandas as pd

from IPython.display import display, HTML

In [2]:
# option to print all the value of cells in DataFrames
pd.set_option("max_colwidth", None)

### Install spacy and download the english pipeline

In [3]:
# install the spacy module
%pip install spacy


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
import spacy
from spacy import displacy

# Esempio di annotazione
Vogliamo stampare le annotazioni sintattiche di una frase.

In [6]:
input_string = "In 1982, Mark drove his car from Los Angeles to Las Vegas until 5 of july"
nlp = spacy.load('en_core_web_sm')

In [30]:
def print_annotation(input_string):
    # esegue il modello del linguaggio sulla stringa in input
    # dal doc possiamo estrarre molte cose, come le frasi, le singole parole, le relazioni ecc.
    doc = nlp(input_string)

    words = []
    # per ogni frase del documento
    for sent in doc.sents:
        # per ogni parola di indice i della frase
        for i, word in enumerate(sent):
            if word.head is word:
                head_idx = 0
            else:
                head_idx = doc[i].head.i+1
            if head_idx == i + 1:
                head_idx = 0

            entity_tag = word.ent_type_
            if len(entity_tag) == 0:
                entity_tag = "O"
            
            word_obj = {
                "id": str(i+1), 
                "word": str(word), 
                "lemma": word.lemma_, 
                "tag": word.tag_, 
                "entity": entity_tag,
                "dependency": word.dep_, 
                "head_id": str(head_idx)
            }
            words.append(word_obj)

    df = pd.DataFrame(words, columns=["id", "word", "lemma", "tag", "entity", "dependency", "head_id"])
    display(df) # stampiamo tutte le dipendenze tra parole, il lemma, il tag, il tipo di entità

- O: significa other. Nella colonna entity vuol dire che non fa parte di nessuna entità;
- Root: nella colonna dependency vuol dire che non ha nessuna dipendenza in entrata;

In [31]:
print_annotation(input_string)

,id,word,lemma,tag,entity,dependency,head_id
0,1,In,in,IN,O,prep,5
1,2,1982,1982,CD,DATE,pobj,1
2,3,",",",",",",O,punct,5
3,4,Mark,Mark,NNP,PERSON,nsubj,5
4,5,drove,drive,VBD,O,ROOT,0
5,6,his,his,PRP$,O,poss,7
6,7,car,car,NN,O,dobj,5
7,8,from,from,IN,O,prep,5
8,9,Los,Los,NNP,GPE,compound,10
9,10,Angeles,Angeles,NNP,GPE,pobj,8


In [32]:
def visualize_annotation(input_string, style="dep"):
    doc = nlp(input_string)
    # Lo stile può essere "dep" per visualizzare le dipendenze tra le parole o "ent" per visualizzare le named entities
    displacy.render(doc, style=style, jupyter=True, options={"distance": 140}) # aumenta la distance per migliorare la leggibilità (default 140)

In [17]:
visualize_annotation(input_string, style="dep")

In [18]:
visualize_annotation(input_string, style="ent") # GPE = Global Position Entity

# Information extraction

Il seguente metodo permette di ottenere informazioni su una singola parola della frase in input.

In [19]:
def get_word_annotation(input_string, word_string):
    doc = nlp(input_string)
    
    words = []
    for sent in doc.sents:
        for i, word in enumerate(sent):
            if word.head is word:
                head_idx = 0
            else:
                head_idx = doc[i].head.i+1
            if head_idx == i + 1:
                head_idx = 0

            entity_tag = word.ent_type_
            if len(entity_tag) == 0:
                entity_tag = "O"
            
            word_obj = {
                "id": i+1,
                "word": str(word), 
                "lemma": word.lemma_, 
                "tag": word.tag_, 
                "entity": entity_tag,
                "dependency": word.dep_, 
                "head id": head_idx
            }
            words.append(word_obj)
    
    for word in words:
        if word["word"] == word_string:
            return word
    
    return None

In [37]:
print(get_word_annotation(input_string, "Mark"))

{'id': 4, 'word': 'Mark', 'lemma': 'Mark', 'tag': 'NNP', 'entity': 'PERSON', 'dependency': 'nsubj', 'head id': 5}


### Esercizio 1: Trovare le relazioni (dependencies)

Definisci un metodo che prende in input una frase (`input_string`) e il nome di una relazione (`relation_string`), analizza l'input con spacy e restituisce le triple (w1, w2, relation) dove w è un lemma. Se la relazione non p presente, restituisce un array vuoto.

```python
def search_relation(input_string, relation_string):
    # your code here...
    return word_obj_list
```

In [20]:
def search_relation(input_string, relation_string):
    word_obj_set = set()
    doc = nlp(input_string)
    words = []
    for sent in doc.sents:
        
        for i, word in enumerate(sent):
            if word.head is word:
                head_idx = 0
            else:
                head_idx = doc[i].head.i+1
            if head_idx == i + 1:
                head_idx = 0

            entity_tag = word.ent_type_
            if len(entity_tag) == 0:
                entity_tag = "O"
            
            word_obj = {
                "id": i+1,
                "word": str(word), 
                "lemma": word.lemma_, 
                "tag": word.tag_, 
                "entity": entity_tag,
                "dependency": word.dep_, 
                "head id": head_idx
            }
            words.append(word_obj)

    for word_obj in words:
        w1 = str(word_obj["word"])
        dep = word_obj["dependency"]
        
        # trova la parola collegata dall'id
        head_id = word_obj["head id"]
        w2 = "-"
        for word_obj in words:
            if head_id == word_obj['id']:
                w2 = str(word_obj['word'])
                break

        # evito le relazioni duplicate
        if (head_id, id, dep) not in word_obj_set and relation_string == dep:
            word_obj_set.add((w1, w2, dep))
    
    return list(word_obj_set)

Ad esempio, cerchiamo tutte le parole in cui una è preposizione dell'altra

In [21]:
search_relation(input_string, "prep")

[('In', 'drove', 'prep'),
 ('to', 'drove', 'prep'),
 ('from', 'drove', 'prep'),
 ('of', '5', 'prep'),
 ('until', 'drove', 'prep')]

### Esercizio 2: Trovare le entità
Definire un metodo che prende in input una frase (`input_string`) e il nome di un tipo di entità (`entity_type_string`), analizza con spacy l'input e restituisce le parole (cioè gli `objects`, non le stringhe) descritti da quella entità. Se il tipo dell'entità non è presente, restituisce un array vuoto.

```python
def search_entity(input_string, entity_type_string):
    return word_obj_list
```

In [91]:
# Vanno restituiti gli object (e.g. "Los Angeles"), non le singole stringhe "Los" e "Angeles"

def search_entity(input_string, ent_label):
    entities_list = []
    doc = nlp(input_string)
    for ent in doc.ents:
        if ent.label_ == ent_label:
            entities_list.append((ent.text, ent.label_))
    return entities_list

In [92]:
entities = search_entity(input_string, "GPE") # GPE | DATE
print(entities)

[('Los Angeles', 'GPE'), ('Las Vegas', 'GPE')]


### Exercise 3: Enriching the sentences

For every sentence in the QuestionClassification dataset, extract the `subject-verb` relation and the `verb-object` relation. Add these couples to the original input, divided by the `#`:

- Sentence: '*What is the full form of .com?*' 
- `subject-verb`: *What is*  
- `verb-object`: *is the full form* 
- Enriched sentence: '*What is the full form of .com? # What is # is the full form*'  

Store the enriched sentences in a new dataframe and train a classifier (SVM, NB, Rocchio..) and evaluate it.

### Exercise 4: Replace texts with entities

For every sentence in the QuestionClassification dataset, extract the entities annotated by the spacy module only for the proper nouns (`PROPN`) and replace the spans in the text with the entity name:

- Sentence: '*In 1982, Mark drove his car from Los Angeles to Las Vegas until 5 of july*'  
- Modified Sentence: '*In 1982, PERSON drove his car from GPE to GPE until 5 of DATE*'

Store the enriched sentences in a new dataframe and train a classifier (SVM, NB, Rocchio..) and evaluate it.

**WARNING**: be careful with `compounds`, they should be replaced by a SINGLE entity name: *Las Vegas* => `GPE`

### Exercise 5: Compare the models

Compare the models from Exercises 3 and 4 with a simpler model you trained in the previous lessons in terms of F1 measure.